# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=12

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==12]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm12', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm12/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-07-16 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_17613/215725244.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_17613/215725244.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_17613/215725244.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
               [,1]     [,2]         [,3]      [,4]
sigmas -0.290405372 1.183439 -0.892765488 4.8153513
       -0.282808718 1.178462 -0.865893807 4.7731020
       -0.275269339 1.173622 -0.839076207 4.7296765
       -0.267786377 1.168921 -0.812330643 4.6851261
       -0.260358995 1.164359 -0.785674494 4.6395020
       -0.252986371 1.159938 -0.759124566 4.5928548
       -0.245667706 1.155659 -0.732697099 4.5452345
       -0.238402215 1.151522 -0.706407767 4.4966905
       -0.231189130 1.147528 -0.680271687 4.4472718
       -0.224027702 1.143677 -0.654303423 4.3970268
       -0.216917196 1.139970 -0.628516988 4.3460033
       -0.209856891 1.136406 -0.602925854 4.2942487
       -0.202846086 1.132986 -0.577542955 4.2418094
       -0.195884090 1.129710 -0.552380693 4.1887317
       -0.188970228 1.126577 -0.527450943 4.1350608
       -0.182103840 1.123587 -0.502765060 4.0808418
       -0.175284278 1.12073

...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]        [,3]     [,4]
sigmas -0.600541423 1.3830605 -2.92168980 4.095432
       -0.592858532 1.3707901 -2.90535062 4.125595
       -0.585234218 1.3585871 -2.88888235 4.155760
       -0.577667594 1.3464520 -2.87228359 4.185918
       -0.570157795 1.3343851 -2.85555304 4.216059
       -0.562703973 1.3223871 -2.83868951 4.246171
       -0.555305299 1.3104584 -2.82169194 4.276245
       -0.547960964 1.2985996 -2.80455938 4.306271
       -0.540670175 1.2868112 -2.78729097 4.336241
       -0.533432157 1.2750937 -2.76988598 4.366144
       -0.526246152 1.2634478 -2.75234375 4.395973
       -0.519111417 1.2518740 -2.73466375 4.425718
       -0.512027227 1.2403728 -2.71684552 4.455372
       -0.504992869 1.2289449 -2.69888868 4.484925
       -0.498007648 1.2175909 -2.68079297 4.514372
       -0.491070882 1.2063114 -2.66255820 4.543703
       -0.484181904 1.1951069 -2.64418424 4.572911
       -0.477340059 1.1839781 -2.62567107 4.6019

        0.196322265 0.5947782  0.73782112 4.980880
        0.199792563 0.5980069  0.75811343 4.936421
        0.203250860 0.6013189  0.77815066 4.891031
        0.206697238 0.6047130  0.79792735 4.844712
        0.210131779 0.6081876  0.81743813 4.797467
        0.213554565 0.6117415  0.83667763 4.749297
        0.216965675 0.6153732  0.85564057 4.700208
        0.220365189 0.6190812  0.87432171 4.650202
        0.223753185 0.6228641  0.89271590 4.599284
        0.227129742 0.6267205  0.91081803 4.547459
        0.230494936 0.6306486  0.92862309 4.494732
        0.233848843 0.6346471  0.94612612 4.441109
        0.237191539 0.6387143  0.96332228 4.386596
        0.240523099 0.6428487  0.98020678 4.331200
        0.243843597 0.6470486  0.99677495 4.274928
        0.247153105 0.6513124  1.01302220 4.217788
        0.250451696 0.6556384  1.02894407 4.159789
        0.253739443 0.6600250  1.04453616 4.100937
        0.257016416 0.6644705  1.05979422 4.041243
        0.260282685 0.6689732  

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_17613/215725244.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_17613/215725244.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_17613/215725244.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

[1] 44.694713  4.722982 66.258718
                [,1]      [,2]         [,3]     [,4]
sigmas -0.4304535170 1.1005721 -2.705970276 4.984986
       -0.4255385361 1.0918634 -2.688183068 4.987094
       -0.4206475942 1.0832189 -2.670506245 4.990406
       -0.4157804571 1.0746376 -2.652930806 4.994887
       -0.4109368944 1.0661187 -2.635447959 5.000504
       -0.4061166788 1.0576615 -2.618049127 5.007223
       -0.4013195862 1.0492653 -2.600725947 5.015010
       -0.3965453959 1.0409293 -2.583470278 5.023832
       -0.3917938902 1.0326530 -2.566274200 5.033656
       -0.3870648546 1.0244358 -2.549130017 5.044449
       -0.3823580776 1.0162771 -2.532030257 5.056177
       -0.3776733506 1.0081764 -2.514967676 5.068809
       -0.3730104679 1.0001333 -2.497935254 5.082311
       -0.3683692269 0.9921474 -2.480926201 5.096652
       -0.3637494275 0.9842183 -2.463933951 5.111799
       -0.3591508725 0.9763457 -2.446952164 5.127723
       -0.3545733675 0.9685293 -2.429974725 5.144391
       -0.35

        0.1285426754 0.5020427  0.463554384 7.215944
        0.1313559333 0.5035972  0.486225515 7.198624
        0.1341612990 0.5052238  0.508795522 7.180352
        0.1369588166 0.5069219  0.531259571 7.161117
        0.1397485300 0.5086911  0.553612755 7.140903
        0.1425304825 0.5105308  0.575850095 7.119700
        0.1453047172 0.5124407  0.597966543 7.097494
        0.1480712768 0.5144201  0.619956983 7.074273
        0.1508302037 0.5164685  0.641816233 7.050025
        0.1535815398 0.5185853  0.663539051 7.024738
        0.1563253269 0.5207698  0.685120134 6.998400
        0.1590616062 0.5230216  0.706554123 6.971000
        0.1617904186 0.5253398  0.727835607 6.942527
        0.1645118050 0.5277238  0.748959124 6.912971
        0.1672258054 0.5301729  0.769919167 6.882319
        0.1699324600 0.5326865  0.790710188 6.850564
        0.1726318084 0.5352636  0.811326599 6.817694
        0.1753238900 0.5379036  0.831762780 6.783702
        0.1780087436 0.5406056  0.852013083 6.

In [2]:
print(error_i)

[2, 3, 6, 7, 8, 9, 10, 11, 12, 14, 15, 17, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 33, 34, 38, 43, 47, 60, 178, 191]
